Installing the main libraries

In [1]:
# !pip install --upgrade tensorflow --user
# !pip install craft-text-detector
# !pip install scikit-learn

Importing the needed libraries

In [6]:
import numpy as np
import cv2
from pathlib import Path
from PIL import Image
import tkinter as tk
from tkinter import filedialog
import pytesseract as tess
tess.pytesseract.tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract.exe"
import os

import re
import pytesseract
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


import cv2
import numpy as np
import string
import pandas as pd
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from collections import Counter
from PIL import Image
from itertools import groupby

from IPython.display import display
from random import randrange

# import craft functions
from craft_text_detector import (
    read_image,
    load_craftnet_model,
    load_refinenet_model,
    get_prediction,
    export_detected_regions,
    export_extra_results,
    empty_cuda_cache
)

Preprocessing and sorting the extracted text

In [7]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key) 

Detecting the text from the image 

In [8]:
def detector_recognizer(image_path):
    data_folder ='Crops'
    output_dir = 'Crop/'
    
    # read image
    image = read_image(image_path)
    # load models
    refine_net = load_refinenet_model(cuda=False)
    craft_net = load_craftnet_model(cuda=False)

    # perform prediction
    prediction_result = get_prediction(
      image=image,
      craft_net=craft_net,
      refine_net=refine_net,
      text_threshold=0.5,
      link_threshold=0.2,
      low_text=0.2,
      #cuda=True,
      long_size=1280
    )

    # export detected text regions
    exported_file_paths = export_detected_regions(
       image=image,
       regions=prediction_result["boxes"],
       output_dir=output_dir,
       rectify=True
    )

    empty_cuda_cache()
    



###################### tesseract  ###########################3

    
#     num = 1000000
#     dirlist = sorted_alphanumeric(os.listdir(data_folder))
    
#     #for path in os.listdir(data_folder):
#     for path in dirlist:
#         #print(data_folder+"/"+path)
#         # Load the image
#         img = cv2.imread(data_folder+"/"+path)

#         # Convert it to the gray-scale
#         gry = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#         pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
#         # OCR detection
#         custom_config = r'--oem 3 --psm 6'

#         d = pytesseract.image_to_data(gry, config=custom_config, output_type=pytesseract.Output.DICT)

#         n_boxes = len(d['level'])

#         for i in range(n_boxes): 
#             if d['text'][i] == '':
#                 continue
#             (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])

#             cv2.rectangle(img, (x, y), (x+w, y+h), (255, 255, 0), 2)

#             # Crop the image
#             crp = gry[y:y+h, x:x+w] 
# #             if '{}'.format(num) == {}:
# #                 cv2.imshow("img", crp)
# #                 cv2.waitKey(0)
# #                 cv2.destroyWindow("img")
# #                 custom_config = r'--oem 3 --psm 10'
# #                 text = tess.image_to_string(crp, config=custom_config, output_type=tess.Output.DICT,lang='eng')
# #                 print("Number  Detected Plate Text : ",text)
#             cv2.imwrite('Crop/image_crops/{}.png'.format(num), crp)
#             print(num)
#             num+=1 

Reading the image

In [51]:
root = tk.Tk()
def open_file_browser():
    # Open a file browser and ask the user to select a file
    file_path = filedialog.askopenfilename()  # Returns the selected file path
    root.withdraw()
    # If a file was selected, do something with the file_path
    if file_path:
        print(f"You selected: {file_path}")
        # Create a Path object and get the suffix
        ext = Path(file_path).suffix

        # Check if the suffix is one of the common image file extensions
        if ext.lower() in ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp']:
            print(f"The file {file_path} is an image file.")
            return file_path
        else:
            print(f"The file {file_path} is not an image file.")
    else:
        print("No file was selected.")


image_path = open_file_browser()
root.destroy()

You selected: C:/Users/Mustafa/Downloads/th (1).jpg
The file C:/Users/Mustafa/Downloads/th (1).jpg is an image file.


In [52]:
detector_recognizer(image_path)

Recognizing the detected text

In [53]:
img_path = "Crop/image_crops/"
imgs_paths = os.listdir("Crop/image_crops")
custom_config = r'--oem 3 --psm 6'
txt_dict = dict()
for path in imgs_paths:
    img = cv2.imread(img_path + path)
    x = img.shape[0]
    y = img.shape[1]
    text = pytesseract.image_to_string(img, config=custom_config, output_type=pytesseract.Output.DICT,lang='eng')
    img1 = cv2.imread(image_path)
    txt = text.get('text')
    txt_dict[txt] = len(txt)
max_key = max(txt_dict, key=txt_dict.get)
#print(type(txt))
# Window name in which image is displayed
window_title = 'Image'

# font
f = cv2.FONT_HERSHEY_SIMPLEX

# point
p = (x//2, y//2)

# font Scale
fScale = 1

# Blue color in BGR
c = (255, 0, 0)

# Line thickness of 2 px
t = 2

# Using cv2.putText() method
image = cv2.putText(img1, max_key, (p), f, 
                   fScale, c, t, cv2.LINE_AA)

# Displaying the image
cv2.imshow(window_title, img1) 
cv2.waitKey(0)
cv2.destroyWindow("Image")
print("Number  Detected Plate Text : ",max_key)

Number  Detected Plate Text :  HR26D05551

